In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


ModuleNotFoundError: ignored

In [ ]:
!pip install --upgrade gensim
!pip install --upgrade pyLDAvis
!pip install --upgrade pandas
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models


불용어 제거

In [ ]:
stopwords_file = 'stopword.txt'
def load_stopwords(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        stopwords = {line.strip() for line in f}
    return stopwords
stop_words = load_stopwords(stopwords_file)

LDA 모델링

In [ ]:

# DataFrame에서 요약 가져오기
df = pd.read_csv('fail2.csv')
summaries = df['요약'].tolist()

# 텍스트 전처리를 위한 함수들 설정
okt = Okt()
stop_words = []

# 텍스트 전처리 함수 정의
def preprocess_text(sentence):
    # 문장에서 형태소와 품사 정보 추출
    tokens_with_pos = okt.pos(sentence)
    # 명사와 형용사만 선택
    tokens = [word for word, pos in tokens_with_pos if pos in ['Noun', 'Adjective']]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# 토픽 수 범위 설정
min_topics = 2
max_topics = 20
step = 2

# 최적의 토픽 수 탐색
best_num_topics = None
best_coherence_score = -1

for num_topics in range(min_topics, max_topics + 1, step):
    # 텍스트 전처리를 거친 요약들의 리스트
    processed_summaries = [preprocess_text(summary) for summary in summaries]

    # 사전과 코퍼스 생성
    dictionary = corpora.Dictionary(processed_summaries)
    corpus = [dictionary.doc2bow(text) for text in processed_summaries]

    # LDA 모델 학습
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)

    # 토픽 일관성 점수 계산
    coherence_model = CoherenceModel(model=lda_model, texts=processed_summaries, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # 최적의 토픽 수 갱신
    if coherence_score > best_coherence_score:
        best_num_topics = num_topics
        best_coherence_score = coherence_score

# 최적의 토픽 수 출력
print(f"Best number of topics: {best_num_topics}")

# 최적의 토픽 수로 LDA 모델 학습
processed_summaries = [preprocess_text(summary) for summary in summaries]
dictionary = corpora.Dictionary(processed_summaries)
corpus = [dictionary.doc2bow(text) for text in processed_summaries]
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=best_num_topics)

# 토픽별로 가장 중요한 단어들을 출력
topics = lda_model.show_topics(num_topics=best_num_topics, num_words=5)
for topic_id, topic_words in topics:
    print(f"Topic #{topic_id + 1}: {topic_words}")

# 각 요약에 대한 토픽 분포 출력
for i, text in enumerate(processed_summaries):
    bow = dictionary.doc2bow(text)
    topic_distribution = lda_model.get_document_topics(bow)
    for topic_id, prob in topic_distribution:
        words = [word for word, prob in lda_model.show_topic


젠심

In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models

data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(data)

워드클라우드

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 텍스트 전처리를 거친 요약들의 리스트
processed_summaries = [preprocess_text(summary) for summary in summaries]

# 토큰을 하나의 리스트로 펼치기
all_tokens = [token for sublist in processed_summaries for token in sublist]

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform([' '.join(text) for text in processed_summaries])

# 토큰의 TF-IDF 가중치 계산
token_weights = {}
for idx, token in enumerate(tfidf.get_feature_names()):
    token_weights[token] = tfidf_matrix[:, idx].mean()

# 가장 높은 TF-IDF 가중치를 가진 상위 10개 토큰 추출
top_tokens = sorted(token_weights.items(), key=lambda x: x[1], reverse=True)[:10]

# 워드 클라우드 생성
wordcloud = WordCloud(width=800, height=400, background_color='white')
wordcloud.generate_from_frequencies(dict(top_tokens))

# 워드 클라우드 시각화
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()